In [3]:
from toqito.channels import partial_trace
from qutip import *
from PIL import Image
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy import linalg
import math
import matplotlib.pyplot as plt
from scipy import optimize
import random
from math import *
import pandas as pd
import random
from mpl_toolkits.mplot3d import Axes3D
from sys import stdout
from tqdm import tqdm
from tqdm import trange
from scipy.linalg import fractional_matrix_power

In [4]:
def state_fidelity(rho_1, rho_2): #fidelity
        if np.shape(rho_1) != np.shape(rho_2):
            print("Dimensions of two states do not match.")
            return 0
        else:
            sqrt_rho_1 = fractional_matrix_power(rho_1, 1 / 2)
            fidelity = np.trace(fractional_matrix_power(sqrt_rho_1 @ rho_2 @ sqrt_rho_1, 1 / 2)) ** 2
            return np.real(fidelity)

In [5]:
def UO(B1,B2,a,D1,D2):
    i   = 1j
    gamma = 2*pi*2.8
    D     = 2870
    UA = [[(B2**2+B1**2*cos(a))/(B1**2+B2**2), -i*B1*(e**(-i*D1))*sin(a)/sqrt(B1**2+B2**2), ((-1+cos(a))*B1*B2*(e**(-i*(D1-D2))))/(B1**2+B2**2)],
            [-i*B1*(e**(i*D1))*sin(a)/sqrt(B1**2+B2**2), cos(a), -i*B2*(e**(i*D2))*sin(a)/sqrt(B1**2+B2**2)],
            [((-1+cos(a))*B1*B2*e**(i*(D1-D2)))/(B1**2+B2**2), -i*B2*(e**(-i*D2))*sin(a)/sqrt(B1**2+B2**2), (B1**2+B2**2*cos(a))/(B1**2+B2**2)]]
    return UA

## Define dimension, pauli matrices
i   = 1j #1j
sx  = 1/sqrt(2)*np.array([[0, 1, 0],[1, 0, 1], [0, 1, 0]])
sy  = 1/sqrt(2)/i*np.array([[0, 1, 0], [-1, 0, 1],[0, -1, 0]])
sz  = np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]])
#sz  = [1, 0, 0; 0, -1, 0; 0, 0, 0]
I   = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Rotation matrix projected into 2 level system
Sxp  = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]])
Sxm  = np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]])
Syp  = 1/i*np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 0]])
Sym  = 1/i*np.array([[0, 0, 0], [0, 0, 1], [0, -1, 0]])
Szp  = np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]])

#Gellman matrix
Sx  = np.array([[0, 0, 1],[0, 0, 0], [1, 0, 0]])
Sy  = np.array([[0, 0, -i],[0, 0, 0], [i, 0, 0]])
Sz  = np.array([[1, 0, 0],[0, 0, 0], [0, 0, -1]])

# Pauli basis for 13C nuclear spin
Ix  = 1/2*np.array([[0, 0, 1], [0, 0, 0], [1, 0, 0]])   
Iy  = 1/2/i*np.array([[0, 0, 1], [0, 0, 0], [-1, 0, 0]])
Iz  = np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]])


## Define sweep parameters
Sweep = 1001
N = Sweep
B = 403 #[G] magnetic field

T = 5; # sweep tau [us]
t = np.linspace(0,T,N)
n = 32; # number of pi pulses

## Define gate operations
# Single Q ms=+1
U090xp = UO(1,0,pi/4,0,0)
U090xmp = UO(1,0,-pi/4,0,0)
U090yp = UO(1,0,pi/4,pi/2,0)
U090ymp = UO(1,0,-pi/4,pi/2,0)
U180xp = UO(1,0,pi/2,0,0)
U180xmp = UO(1,0,-pi/2,0,0)

#Single Q ms=-1
U090xm = UO(0,1,pi/4,0,0)
U090xmm = UO(0,1,-pi/4,0,0)
U180xm = UO(0,1,pi/2,0,0)
U180xmm = UO(0,1,pi/2,0,0)

# Define initial state of the system

irho_p = np.array([[1,0,0],[0,0,0],[0,0,0]]) #[0,0,0;0,0,0]

irho_m = np.array([[0,0,0],[0,0,0],[0,0,1]]) #[0,0,0;0,0,1]

irho_z = np.array([[0,0,0],[0,1,0],[0,0,0]]) #[0,1,0;0,0,0]

irho_mix = np.array([[1/2,0,0],[0,1/2,0],[0,0,0]]) #[1/2,0,0;0,1/2,0;0,0,0]

irho_Z = np.array([[0,0,0],[0,0,0],[0,0,1]]) #target state

irho_MIX = np.array([[1/2,0,0],[0,0,0],[0,0,1/2]])

In [6]:
arr1 = []

In [7]:
irho = np.kron(irho_z,irho_MIX) #initial state
print(irho)

[[0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]]


In [8]:
trace = [1, 1, 0, 100, 100, 1000]
vvv = [0, 0, 0, 0]
normalxyz = [0, 0, 0]

#for making 13C nuclear random dataset
gammaN = 2*pi*1.071e-3 #[MHz/G]
# Al    = 2*pi * random.uniform(0.05, 0.8) #[MHz] # A_|| hyperfine term
# Ap = 2*pi* random.uniform(0.05, 0.3) #[MHz] # A_per hyperfine term

Al = 1.4758600370352584
Ap = 1.3393631030877522

#Initialization
rho_0 = (np.kron(U090xp,I))@irho@((np.kron(U090xp,I)).conj().T) # superposition state on NV

Sa= []

ham = Al*np.kron(sz,Iz) + Ap*np.kron(sz,Ix) + B*gammaN*np.kron(I,Iz) # Hamiltonian
eigvals = np.linalg.eigh(ham)[0]            # diagonalizing the Hamiltonian 여기서부터 문제 
eigvecs = -1*np.linalg.eigh(ham)[1]         # eigenvectors
E = np.diag(eigvals)                        # exponent of eigenvalues
U_H= eigvecs.conj().T                       # unitary matrix formed by eigenvectors

In [9]:
udf = pd.DataFrame(U_H)
udf

,0,1,2,3,4,5,6,7,8
0,0.079203,-0.0,-0.996859,0.0,0.0,0.0,0.000000,0.0,-0.000000
1,-0.000000,-0.0,-0.000000,-0.0,-0.0,-1.0,-0.000000,-0.0,-0.000000
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.245718,-0.0,0.969341
3,-0.000000,-0.0,-0.000000,-0.0,-1.0,-0.0,-0.000000,-0.0,-0.000000
4,-0.000000,1.0,-0.000000,-0.0,-0.0,-0.0,-0.000000,-0.0,-0.000000
5,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.000000,-1.0,-0.000000
6,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.0,0.969341,-0.0,-0.245718
7,-0.000000,-0.0,-0.000000,-1.0,-0.0,-0.0,-0.000000,-0.0,-0.000000
8,0.996859,-0.0,0.079203,-0.0,-0.0,-0.0,-0.000000,-0.0,-0.000000


In [10]:
rho1 = np.kron(U090yp,I)@irho@(np.kron(U090yp,I).conj().T)                              # Ry 90도
df1 = pd.DataFrame(rho1)
df1 

,0,1,2,3,4,5,6,7,8
0,0.25-0.00j,0.0+0.0j,0.00+0.00j,-0.25-0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
1,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
2,0.00+0.00j,0.0+0.0j,0.25-0.00j,0.00+0.00j,0.0+0.0j,-0.25-0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
3,-0.25+0.00j,0.0+0.0j,0.00+0.00j,0.25+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
4,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
5,0.00+0.00j,0.0+0.0j,-0.25+0.00j,0.00+0.00j,0.0+0.0j,0.25+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
6,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
7,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
8,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j


In [11]:
vari=[1.3580825388312319,10.0,0.16800675840612264,22.0]  #초기값

In [12]:
U_e2=(U_H.conj().T)@(linalg.expm(-i*E* vari[0]/2)@U_H)                                  # for tau/2
U_e=(U_H.conj().T)@(linalg.expm(-i*E* vari[0])@U_H)                                     # for tau
rho2=U_e2@rho1@(U_e2.conj().T)                                                          # first tau/2
for k in range(1,2*math.trunc(vari[1])):                                                # N과 tau를 N개 생성
    rho2 = U_e@np.kron(U180xp,I) @ rho2 @ (np.kron(U180xp,I).conj().T) @ (U_e.conj().T) # N & tau
rho3 = U_e2 @ np.kron(U180xp,I) @ rho2 @ (np.kron(U180xp,I).conj().T) @ (U_e2.conj().T) # last N & tau/2

In [13]:
# df2 = pd.DataFrame(rho2)
df2 = pd.DataFrame(rho3)
df2

,0,1,2,3,4,5,6,7,8
0,2.500000e-01+4.212738e-17j,0.0+0.0j,-1.841088e-16+5.867996e-17j,-3.801912e-03+2.607474e-03j,0.0+0.0j,-7.520787e-02-2.383748e-01j,0.0+0.0j,0.0+0.0j,0.0+0.0j
1,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
2,-2.098796e-16-7.128637e-17j,0.0+0.0j,2.500000e-01+3.293638e-17j,7.520787e-02-2.383748e-01j,0.0+0.0j,-3.801912e-03-2.607474e-03j,0.0+0.0j,0.0+0.0j,0.0+0.0j
3,-3.801912e-03-2.607474e-03j,0.0+0.0j,7.520787e-02+2.383748e-01j,2.500000e-01+5.950829e-17j,0.0+0.0j,1.821198e-16-9.454182e-17j,0.0+0.0j,0.0+0.0j,0.0+0.0j
4,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
5,-7.520787e-02+2.383748e-01j,0.0+0.0j,-3.801912e-03+2.607474e-03j,1.911548e-16+9.646639e-17j,0.0+0.0j,2.500000e-01+4.505820e-17j,0.0+0.0j,0.0+0.0j,0.0+0.0j
6,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
7,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
8,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.0+0.0j,0.0+0.0j


In [14]:
#for e Rx(pi/2)
rho4 = np.kron(U090xp,I)@rho3@(np.kron(U090xp,I).conj().T)                              # Rx 90도
df4 = pd.DataFrame(rho4)
print(df4)

                            0         1                           2  \
0  2.473925e-01+5.972405e-17j  0.0+0.0j  2.383748e-01-7.520787e-02j   
1  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j   
2  2.383748e-01+7.520787e-02j  0.0+0.0j  2.526075e-01+4.179386e-17j   
3 -3.801912e-03+4.326093e-17j  0.0+0.0j  7.781992e-17+1.638058e-16j   
4  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j   
5 -5.090618e-17+1.523091e-16j  0.0+0.0j -3.801912e-03+4.010316e-17j   
6  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j   
7  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j   
8  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j   

                            3         4                           5         6  \
0 -3.801912e-03-5.972931e-17j  0.0+0.0j -7.582201e-17-2.128670e-16j  0.0+0.0j   
1  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j  0.0+0.0j   
2  1.166135e-16-2.521193e-16j  0.0+0.0j -3.801